### Authenticate

In [1]:
# authenticate the logged in user in Colab towards GCP
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# Find project ids using gcloud and parse the result table into an array of ids
projects_list = !gcloud projects list
projects_list.pop(0)
#project_ids = ['PROJ-A','PROJ-B','...']
project_ids = []

if len(project_ids) == 0:
  for project in projects_list:
    a = project.replace(" ", ",")
    b = a.split(",")
    project_ids.append(b[0])

print(project_ids)

In [ ]:
# declare variables for the destination tables where usage will be persisted
project_id = 'PROJECT_ID' #project id where temporary table is written
dataset_id = 'DATASET_ID' #dataset where temporary table is created
table_id = 'slot_usage'
start_date = '2024-01-01' #start date for collecting BigQuery usage
region = 'REGION' #region such as region-eu, region-us, europe-west1 where the jobs have taken place

import pandas as pd
import pandas_gbq
from google.cloud import bigquery

df = pd.DataFrame()

# loop through all project ids to retrieve usage from INFORMATION_SCHEMA table (source: https://docs.google.com/document/d/1mUFlH-seavSpPob1K9O7yz3SJ_MElI6zVt5yqyMJRco/edit?resourcekey=0-GYbZ5Tg1UD67KzkTHaMqtw#heading=h.vklkfaoltjc7)
for pid in project_ids:
  try:
    client = bigquery.Client(project=pid)
    query = f"""
WITH
  pay_as_go_total AS (
  SELECT
    SUM(total_bytes_billed) AS total_bytes_billed
  FROM
    `{region}`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_PROJECT
  WHERE
    DATE(period_start,'Europe/Oslo') > CAST('{start_date}' AS date)
    AND (statement_type != 'SCRIPT'
      OR statement_type IS NULL)
    AND period_slot_ms > 0
    AND job_type = 'QUERY'
    AND state = 'DONE' ),
  snapshot_data AS (
  SELECT
    project_id,
    period_start,
    period_slot_ms
  FROM
    `{region}`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_PROJECT
  WHERE
    DATE(period_start,'Europe/Oslo') > CAST('{start_date}' AS date)
    AND (statement_type != 'SCRIPT'
      OR statement_type IS NULL)
    AND period_slot_ms > 0
    AND job_type = 'QUERY' ),
  snapshot_data_with_bytes AS (
  SELECT
    *
  FROM
    snapshot_data
  JOIN
    pay_as_go_total
  ON
    TRUE ),
  data_by_time AS (
  SELECT
    project_id,
    TIMESTAMP_TRUNC(period_start, SECOND ) AS usage_time_sec,
    IFNULL(SUM(period_slot_ms) / 1000, 0) AS usage_slot_sec,
    total_bytes_billed
  FROM
    snapshot_data_with_bytes
  GROUP BY
    usage_time_sec,
    project_id,
    total_bytes_billed),
  max_per_minute_data_by_time AS (
  SELECT
    project_id,
    TIMESTAMP_TRUNC(usage_time_sec, MINUTE ) AS usage_time,
    MAX(usage_slot_sec) AS max_slot_per_minute,
    total_bytes_billed
  FROM
    data_by_time
  GROUP BY
    usage_time,
    project_id,
    total_bytes_billed ),
  max_per_minute_data_by_time_with_zeros AS (
  SELECT
    project_id,
    usage_time,
    IFNULL(slot_usage.max_slot_per_minute,0) AS max_slot_per_minute,
    total_bytes_billed
  FROM
    UNNEST(GENERATE_TIMESTAMP_ARRAY('{start_date}', CURRENT_TIMESTAMP(), INTERVAL 1 MINUTE)) AS usage_time
  LEFT JOIN (
    SELECT
      *
    FROM
      max_per_minute_data_by_time ) AS slot_usage
  USING
    (usage_time))
SELECT
  *
FROM
  max_per_minute_data_by_time_with_zeros
ORDER BY
  usage_time asc
"""
    dfx = client.query(query).to_dataframe()
    df = pd.concat([df, dfx])
  except:
    print(f"An error occurred with {pid}")

# write back the result to BigQuery: {project_id}.{dataset_id}.{table_id}'
pandas_gbq.to_gbq(df, f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')
print('Sample table format below:')
df.head(5)

In [ ]:
# define variables to match the current region ()
earliest_date = ''
if earliest_date == '':
  earliest_date = df['usage_time'].min()

latest_date = ''
if latest_date == '':
  latest_date = df['usage_time'].max()

committed_slots = 100
std_ed_price_per_slot = 0.044
ent_ed_price_per_slot = 0.066
ent_ed_1yc_per_slot = 0.0528
ent_ed_3yc_per_slot = 0.0396
ent_edplus_price_per_slot = 0.11
ent_edplus_1yc_per_slot = 0.088
ent_edplus_3yc_per_slot = 0.066

# calculate costs per edition and commit duration, as well as pay-as-you-go
# pricing data is extracted from https://cloud.google.com/bigquery/pricing#capacity_compute_analysis_pricing
# Note: with BigQuery editions, slots scale by multiples of 100 (https://cloud.google.com/bigquery/docs/slots-autoscaling-intro#use_autoscaling_reservations)
client = bigquery.Client(project=project_id)
query = f"""
DECLARE
  committed_slots INT64 DEFAULT {committed_slots};
DECLARE
  std_ed_price_per_slot FLOAT64 DEFAULT ({std_ed_price_per_slot}/60);
DECLARE
  ent_ed_price_per_slot FLOAT64 DEFAULT ({ent_ed_price_per_slot}/60);
DECLARE
  ent_ed_1yc_per_slot FLOAT64 DEFAULT ({ent_ed_1yc_per_slot}/60);
DECLARE
  ent_ed_3yc_per_slot FLOAT64 DEFAULT ({ent_ed_3yc_per_slot}/60);
DECLARE
  ent_edplus_price_per_slot FLOAT64 DEFAULT ({ent_edplus_price_per_slot}/60);
DECLARE
  ent_edplus_1yc_per_slot FLOAT64 DEFAULT ({ent_edplus_1yc_per_slot}/60);
DECLARE
  ent_edplus_3yc_per_slot FLOAT64 DEFAULT ({ent_edplus_3yc_per_slot}/60);

WITH
  pay_as_go_tot as (
    SELECT round(sum(distinct(total_bytes_billed))/1e+12, 2) as tot_bytes_billed_TB FROM `{project_id}.{dataset_id}.{table_id}` where usage_time > '{earliest_date}' and usage_time < '{latest_date}' AND project_id IS NOT NULL
  ),
  concat_slots AS (
  SELECT
    usage_time,
    SUM((floor((ceiling(max_slot_per_minute) + 99) / 100) * 100)) AS tot_max_slot_per_minute,
    SUM(max_slot_per_minute) AS sum_max_slot_per_minute
  FROM
    `{project_id}.{dataset_id}.{table_id}`
  WHERE
    usage_time > '{earliest_date}' and usage_time < '{latest_date}'
    AND project_id IS NOT NULL
  GROUP BY
    usage_time),
    price_per_min_bucket as (
SELECT
  *,
  tot_max_slot_per_minute*std_ed_price_per_slot AS cost_std_ed_price_per_slot,
  tot_max_slot_per_minute*ent_ed_price_per_slot AS cost_ent_ed_price_per_slot,
  CASE
    WHEN tot_max_slot_per_minute > committed_slots THEN ((tot_max_slot_per_minute-committed_slots)*ent_ed_price_per_slot)+(committed_slots*ent_ed_1yc_per_slot)
    WHEN tot_max_slot_per_minute <= committed_slots THEN (committed_slots*ent_ed_1yc_per_slot)
  END
  AS cost_ent_ed_1yc_per_slot,
  CASE
    WHEN tot_max_slot_per_minute > committed_slots THEN ((tot_max_slot_per_minute-committed_slots)*ent_ed_price_per_slot)+(committed_slots*ent_ed_3yc_per_slot)
    WHEN tot_max_slot_per_minute <= committed_slots THEN (committed_slots*ent_ed_3yc_per_slot)
  END
  AS cost_ent_ed_3yc_per_slot,
  tot_max_slot_per_minute*ent_edplus_price_per_slot AS cost_ent_edplus_price_per_slot,
  CASE
    WHEN tot_max_slot_per_minute > committed_slots THEN ((tot_max_slot_per_minute-committed_slots)*ent_edplus_price_per_slot)+(committed_slots*ent_edplus_1yc_per_slot)
    WHEN tot_max_slot_per_minute <= committed_slots THEN (committed_slots*ent_edplus_1yc_per_slot)
  END
  AS cost_ent_edplus_1yc_per_slot,
  CASE
    WHEN tot_max_slot_per_minute > committed_slots THEN ((tot_max_slot_per_minute-committed_slots)*ent_edplus_price_per_slot)+(committed_slots*ent_edplus_3yc_per_slot)
    WHEN tot_max_slot_per_minute <= committed_slots THEN (committed_slots*ent_edplus_3yc_per_slot)
  END
  AS cost_ent_edplus_3yc_per_slot
FROM
  concat_slots)
SELECT
  '{earliest_date}' as date_from,
  '{latest_date}' as date_to,
  {committed_slots} as committed_slots,
  cast(avg(sum_max_slot_per_minute) as INT64) AS avg_max_slots_per_billed_minute,
  '----------------------------------' as input_pricing,
  {std_ed_price_per_slot} as std_ed_price_per_slot_hr,
  {ent_ed_price_per_slot} as ent_ed_price_per_slot_hr,
  {ent_ed_1yc_per_slot} as ent_ed_1yc_per_slot_hr,
  {ent_ed_3yc_per_slot} as ent_ed_3yc_per_slot_her,
  {ent_edplus_price_per_slot} as ent_edplus_price_per_slot_hr,
  {ent_edplus_1yc_per_slot} as ent_edplus_1yc_per_slot_hr,
  {ent_edplus_3yc_per_slot} as ent_edplus_3yc_per_slot_hr,
  count(*) as num_mins_billed,
  TIMESTAMP_DIFF('{latest_date}', '{earliest_date}', MINUTE) AS num_mins_total,
  '----------------------------------' as on_demand_pricing,
  tot_bytes_billed_TB as total_TB_ondemand,
  round((tot_bytes_billed_TB * 6.25), 2) as cost_USD_ondemand,
  '----------------------------------' as capacity_pricing,
  round(sum(cost_std_ed_price_per_slot), 2) as cost_USD_std_ed_no_commmit,
  round(sum(cost_ent_ed_price_per_slot), 2) as cost_USD_ent_ed_no_commit,
  round(sum(cost_ent_ed_1yc_per_slot)+((TIMESTAMP_DIFF('{latest_date}', '{earliest_date}', MINUTE)-count(*))*ent_ed_1yc_per_slot*{committed_slots}), 2) as cost_USD_ent_ed_1y_commit,
  round(sum(cost_ent_ed_3yc_per_slot)+((TIMESTAMP_DIFF('{latest_date}', '{earliest_date}', MINUTE)-count(*))*ent_ed_3yc_per_slot*{committed_slots}), 2) as cost_USD_ent_ed_3y_commit,
  round(sum(cost_ent_edplus_price_per_slot), 2) as cost_USD_ent_edplus_no_commit,
  round(sum(cost_ent_edplus_1yc_per_slot)+((TIMESTAMP_DIFF('{latest_date}', '{earliest_date}', MINUTE)-count(*))*ent_edplus_1yc_per_slot*{committed_slots}), 2) as cost_USD_ent_edplus_1y_commit,
  round(sum(cost_ent_edplus_3yc_per_slot)+((TIMESTAMP_DIFF('{latest_date}', '{earliest_date}', MINUTE)-count(*))*ent_edplus_3yc_per_slot*{committed_slots}), 2) as cost_USD_ent_edplus_3y_commit
FROM
  price_per_min_bucket
JOIN
  pay_as_go_tot
ON TRUE
where usage_time >= '{earliest_date}'
group by tot_bytes_billed_TB
"""
dfx = client.query(query).to_dataframe()
print(dfx.transpose())


In [ ]:
# group max_slots_per_minute per 100 slots buckets. The first bucket represents 0 > 100 slots, the second 101 > 200 slots, etc
client = bigquery.Client(project=project_id)
query = f"""
WITH
  all_buckets AS (
  SELECT
    GENERATE_ARRAY(0, MAX(max_slot_per_minute) + 100, 100) AS bucket
  FROM
    `{project_id}.{dataset_id}.{table_id}`
  WHERE
    usage_time >= '{earliest_date}'
    AND project_id IS NOT NULL )
SELECT
  bucket,
  COUNTIF(max_slot_per_minute <= bucket + 100 AND max_slot_per_minute > bucket) AS minutes,
FROM
  `{project_id}.{dataset_id}.{table_id}`,
  UNNEST((
    SELECT
      *
    FROM
      all_buckets)) AS bucket
WHERE
  usage_time >= '{earliest_date}'
  AND project_id IS NOT NULL
GROUP BY
  bucket
"""
dfx = client.query(query).to_dataframe()
print(dfx)